In [1]:
BUILD_DIR=/workdir/build
TARGET_DIR=$BUILD_DIR/icu4c-target
LIB_DIR=$(zig env | jq -r '.lib_dir')

In [2]:
cd /workdir/build/icu4c-source/source

In [3]:
ls /workdir/build/icu4c-source/source

Doxyfile.in   common         configure.ac   io               stubdata
Makefile      config         data           layoutex         test
Makefile.in   config.guess   extra          lib              tools
acinclude.m4  config.log     i18n           mkinstalldirs
aclocal.m4    config.status  icudefs.mk     python
allinone      config.sub     icudefs.mk.in  runConfigureICU
bin           configure      install-sh     samples


In [4]:
make distclean || make clean
rm -Rf /workdir/build/icu4c-target/linux-x86

make[0]: Making `distclean' in `stubdata'
make[1]: Entering directory '/workdir/build/icu4c-source/source/stubdata'
test -z "*~ stubdata.d   " || rm -rf *~ stubdata.d   
rm -rf stubdata.o stubdata.ao libicudata.a 
rm -rf Makefile
make[1]: Leaving directory '/workdir/build/icu4c-source/source/stubdata'
make[0]: Making `distclean' in `common'
make[1]: Entering directory '/workdir/build/icu4c-source/source/common'
test -z "*~ appendable.d bmpset.d brkeng.d brkiter.d bytesinkutil.d bytestream.d bytestrie.d bytestriebuilder.d bytestrieiterator.d caniter.d characterproperties.d chariter.d charstr.d cmemory.d cstr.d cstring.d cwchar.d dictbe.d dictionarydata.d dtintrv.d edits.d emojiprops.d errorcode.d filteredbrk.d filterednormalizer2.d icudataver.d icuplug.d loadednormalizer2impl.d localebuilder.d localematcher.d localeprioritylist.d locavailable.d locbased.d locdispnames.d locdistance.d locdspnm.d locid.d loclikely.d loclikelysubtags.d locmap.d locresdata.d locutil.d lsr.d lstmbe.d message

In [5]:
ls -la /workdir/build/icu4c-target/

total 8
drwxr-xr-x 2 ubuntu ubuntu 4096 Aug  8 14:37 .
drwxr-xr-x 4 ubuntu ubuntu 4096 Aug  7 06:52 ..


In [6]:
OS=linux
ARC=x86

export SYSROOT=$LIB_DIR/libc
export CC="zig cc -target x86_64-linux-musl --sysroot $SYSROOT"
export CXX="zig c++ -target x86_64-linux-musl --sysroot $SYSROOT"

TARGET="$TARGET_DIR/${OS}-${ARC}"
mkdir -p "$TARGET"
./configure                    \
  --prefix="$TARGET"           \
  --enable-static              \
  --disable-shared             \
  --with-data-packaging=static \
  --disable-samples            \
  --disable-tests

echo
make -j$(nproc)
make install

checking for ICU version numbers... release 77.1, library 77.1, unicode version 16.0
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking whether to build debug libraries... no
checking whether to build release libraries... yes
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C... yes
checking whether zig cc -target x86_64-linux-musl --sysroot /opt/zig/lib/libc accepts -g... yes
checking for zig cc -target x86_64-linux-musl --sysroot /opt/zig/lib/libc option to enable C11 features... none needed
checking whether the compiler supports GNU C++... yes
checking whether zig c++ -target x86_64-linux-musl --sysroot /opt/zig/lib/libc accepts -g... yes
checking for zig c++ -target x86_64-linux-musl --sysroot /opt/zig/lib/

In [7]:
ls -la $TARGET

total 28
drwxr-xr-x 7 ubuntu ubuntu 4096 Aug  8 14:37 .
drwxr-xr-x 3 ubuntu ubuntu 4096 Aug  8 14:37 ..
drwxr-xr-x 2 ubuntu ubuntu 4096 Aug  8 14:37 bin
drwxr-xr-x 3 ubuntu ubuntu 4096 Aug  8 14:37 include
drwxr-xr-x 4 ubuntu ubuntu 4096 Aug  8 14:37 lib
drwxr-xr-x 2 ubuntu ubuntu 4096 Aug  8 14:37 sbin
drwxr-xr-x 4 ubuntu ubuntu 4096 Aug  8 14:37 share


In [8]:
ls -la $TARGET/lib

total 103280
drwxr-xr-x 4 ubuntu ubuntu     4096 Aug  8 14:37 .
drwxr-xr-x 7 ubuntu ubuntu     4096 Aug  8 14:37 ..
drwxr-xr-x 3 ubuntu ubuntu     4096 Aug  8 14:37 icu
-rwxr-xr-x 1 ubuntu ubuntu 31896372 Aug  8 14:37 libicudata.a
-rwxr-xr-x 1 ubuntu ubuntu 44991276 Aug  8 14:37 libicui18n.a
-rwxr-xr-x 1 ubuntu ubuntu   870762 Aug  8 14:37 libicuio.a
-rwxr-xr-x 1 ubuntu ubuntu   600682 Aug  8 14:37 libicutest.a
-rwxr-xr-x 1 ubuntu ubuntu  2804288 Aug  8 14:37 libicutu.a
-rwxr-xr-x 1 ubuntu ubuntu 24565676 Aug  8 14:37 libicuuc.a
drwxr-xr-x 2 ubuntu ubuntu     4096 Aug  8 14:37 pkgconfig


In [9]:
for f in $(ls $TARGET/lib/*.a) ; do
    nm $f | head -n 10
    if [ $(nm "$f" | wc -l) -gt 10 ]; then
      echo "..."
    fi
done
echo 


icudt77l_dat.o:
0000000000000000 R icudt77_dat

alphaindex.ao:
0000000000000000 r GCC_except_table15
0000000000000000 r GCC_except_table16
0000000000000000 r GCC_except_table17
0000000000000000 r GCC_except_table19
0000000000000000 r GCC_except_table21
0000000000000000 r GCC_except_table22
0000000000000000 r GCC_except_table24
0000000000000000 r GCC_except_table27
...

locbund.ao:
0000000000000000 r GCC_except_table2
                 U _Unwind_Resume
0000000000000000 t _ZL15locbund_cleanupv
0000000000000000 b _ZL18gPosixNumberFormat
                 U _ZSt9terminatev
0000000000000000 b _ZZL22copyInvariantFormatterP13ULocaleBundle18UNumberFormatStyleE5gLock
0000000000000000 W __clang_call_terminate
                 U __cxa_begin_catch
...

ctest.ao:
0000000000000010 r .LCPI2_0
0000000000000000 r .LCPI2_1
0000000000000028 r .LCPI2_2
0000000000000020 r .LCPI2_3
0000000000000030 r .LCPI2_4
0000000000000018 r .LCPI2_5
0000000000000008 r .LCPI5_0
0000000000000030 r .LCPI5_1
...

collationin